Step 1: download and convert dataset from CSV to Parquet

In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
from dask.distributed import Client, LocalCluster
import dask
from dask.distributed import get_task_stream

In [3]:
import time

In [4]:
print('pandas version: %s' % pd.__version__)
print('numpy version: %s' % np.__version__)
print('dask version: %s' % dask.__version__)

pandas version: 1.3.4
numpy version: 1.20.3
dask version: 2022.04.0


In [5]:
print('dask distributed version: %s' % dask.distributed.__version__)

dask distributed version: 2022.2.1+72.g25f1d481


https://docs.databricks.com/_static/notebooks/koalas-benchmark-distributed-execution.html?_ga=2.216403934.95291449.1648935555-599276868.1645477063

In [ ]:
filename = "taxi_dataset.txt"

In [ ]:
with open(filename) as file:
    csv_files = [line.rstrip() for line in file]
# only choose yellow taxis
yellow = list(filter(lambda x: "yellow" in x, csv_files))

In [ ]:
# make the list small for now
yellow = ['https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-11.csv']

In [ ]:
yellow

In [ ]:
import re
pattern = '/[^\/]*\.csv$/gm'

for csv_url in yellow:
#     get the filename only so I can sort by month / year
    csv_name = re.findall(r"[^\/]*\.csv$",csv_url)
    df = dd.read_csv(csv_url,dtype={'tolls_amount': 'float64'})

    df = df.repartition(npartitions=4)
    df.to_parquet(f'./tmp/trip_data_{csv_name}', write_index=False)


Read in parquet to dask

In [4]:
from dask.distributed import get_task_stream

client = Client()

# set up testbench

In [ ]:
def benchmark(f, df, benchmarks, task_name, **kwargs):
    """Benchmark the given function against the given DataFrame.
    
    Parameters
    ----------
    f: function to benchmark
    df: data frame
    benchmarks: container for benchmark results
    name: task name
    
    Returns
    -------
    Duration (in seconds) of the given operation
    """
    ret_benchmark_vals = {}
    with get_task_stream(plot='save', filename="task-stream.html") as ts:
        start_time = time.time()
        ret = f(df, **kwargs)
        ret_benchmark_vals['raw_duration'] = time.time() - start_time
        ret_benchmark_vals['history'] = ts.data
    benchmarks[task_name] = ret_benchmark_vals
    print(f"{task_name} took: {benchmarks[task_name].get('raw_duration')} seconds")
    return benchmarks[task_name].get("raw_duration")

In [ ]:
import collections
dask_benchmarks = collections.defaultdict(dict)
# benchmarks = {"task1" : {"stat1": val, "stat2": val}}

# Define benchmark tasks

In [ ]:
all_tasks = []

In [ ]:
# sum, then means = simple mapreduce
def read_to_basic_ETL(df = None):
    df = dd.read_parquet(
    "./tmp/", 
    storage_options={"anon": True, 'use_ssl': True})    
    return (df.fare_amount + df.tip_amount).mean().compute()
all_tasks.append(read_to_basic_ETL)

In [ ]:
# counts of values seen = simple map, groupby, reduce
def count_values(df):
    return df.fare_amount.value_counts().compute()
all_tasks.append(count_values)

In [ ]:
# cpu heavy arithmetic : mapreduce
def complicated_arithmetic_operation(df):
    theta_1 = df.pickup_longitude
    phi_1 = df.pickup_latitude
    theta_2 = df.dropoff_longitude
    phi_2 = df.dropoff_latitude
    temp = (np.sin((theta_2-theta_1)/2*np.pi/180)**2
           + np.cos(theta_1*np.pi/180)*np.cos(theta_2*np.pi/180) * np.sin((phi_2-phi_1)/2*np.pi/180)**2)
    ret = 2 * np.arctan2(np.sqrt(temp), np.sqrt(1-temp))
    return ret.compute()
all_tasks.append(complicated_arithmetic_operation)

In [ ]:
def groupby_statistics(df):
    return df.groupby(by='passenger_count').agg(
      {
        'total_amount': ['mean', 'std'], 
        'tip_amount': ['mean', 'std']
      }
    ).compute()
all_tasks.append(complicated_arithmetic_operation)

In [ ]:
# join two datasets
def join_data(df):
    return dd.merge(df, other, left_index=True, right_index=True).compute()
all_tasks.append(complicated_arithmetic_operation)

# run the tasks

In [5]:
df = dd.read_parquet(
    "./tmp/", 
    storage_options={"anon": True, 'use_ssl': True})

In [ ]:
# for task in all_tasks:
#     benchmark(task, df=df, benchmarks = dask_benchmarks, task_name = task.__name__)

In [6]:
from scheduler_profilers import pyspy_on_scheduler, viztrace_scheduler


In [8]:
with (
    pyspy_on_scheduler("pyspy2.json"),
    # ^ Saves a speedscope profile to `pyspy.json` locally
    viztrace_scheduler(
        "viztracer.json", trace_sparse="distributed.Scheduler.update_graph_hlg"
    ),
    # ^ Saves a Chrome trace to `viztracer.json` locally
):
    df.fare_amount.value_counts().compute()

2022-04-22 19:15:47,523 - distributed.core - ERROR - update_graph_hlg() missing 1 required positional argument: 'self'
Traceback (most recent call last):
  File "/Users/mk/projects/distributed/distributed/core.py", line 625, in handle_stream
    handler(**merge(extra, msg))
TypeError: update_graph_hlg() missing 1 required positional argument: 'self'
2022-04-22 19:15:47,527 - distributed.core - ERROR - Exception while handling op register-client
Traceback (most recent call last):
  File "/Users/mk/projects/distributed/distributed/core.py", line 559, in handle_comm
    result = await result
  File "/Users/mk/projects/distributed/distributed/scheduler.py", line 5498, in add_client
    await self.handle_stream(comm=comm, extra={"client": client})
  File "/Users/mk/projects/distributed/distributed/core.py", line 625, in handle_stream
    handler(**merge(extra, msg))
TypeError: update_graph_hlg() missing 1 required positional argument: 'self'
2022-04-22 19:15:47,529 - tornado.application - E

Saving report to /var/folders/jn/54vtqbx5231b0w09shgckwv80000gn/T/tmpz5a1_29yviztracer..json ...Loading finish                                        

Dumping trace data to json, total entries: 2, estimated json file size: 240.0B
Report saved.


CancelledError: ('value-counts-agg-6077da6b9ab31c4748e89eb4ef5c072d', 0)

In [ ]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

In [ ]:
df.fare_amount.value_counts().compute()

In [ ]:
[benchmark(task, df=df, benchmarks = dask_benchmarks, task_name = task.__name__) for task in all_tasks]

# history groking

In [ ]:
# add the analyzed dataframes

for task_name, output_values in dask_benchmarks.items():
    dask_hx = output_values.get("history")
    hx_df = pd.DataFrame (dask_hx, columns = ['worker','status','nbytes', 'thread', 'type', 'typename', 'metadata', 'startstops', 'key'])
    hx_ddf = dd.from_pandas(hx_df, npartitions=1)
    exploded_df = hx_ddf.explode("startstops")
    exploded_df['action'] = exploded_df['startstops'].apply(lambda x: x['action'], meta = ("action", str))
    exploded_df['start'] = exploded_df['startstops'].apply(lambda x: x['start'], meta = ("start", np.float64))
    exploded_df['end'] = exploded_df['startstops'].apply(lambda x: x['stop'], meta = ("stop", np.float64))
    exploded_df['action_duration'] = exploded_df['end'] - exploded_df['start']
    exploded_df_only_agg_fields = exploded_df[['worker', 'action', 'action_duration']]
    time_per_worker_and_action = exploded_df_only_agg_fields.groupby(['worker','action']).agg("sum")
    nbytes_per_worker = hx_ddf[['worker', 'nbytes']].groupby(["worker"]).agg("sum")
    output_values["time_per_worker_and_action"] = time_per_worker_and_action.compute()
    output_values["nbytes_per_worker"] = nbytes_per_worker.compute()

In [ ]:
# access the analyzed dataframes like so:
dask_benchmarks['read_to_basic_ETL']["time_per_worker_and_action"]
dask_benchmarks['read_to_basic_ETL']["nbytes_per_worker"]

# try to do something fancy with the history

In [ ]:
hx = dask_benchmarks.get("read_to_basic_ETL").get("history")

use dask to do dask :D

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [ ]:
hx_df = pd.DataFrame (hx, columns = ['worker','status','nbytes', 'thread', 'type', 'typename', 'metadata', 'startstops', 'key'])

In [ ]:
hx_ddf = dd.from_pandas(hx_df, npartitions=1)

In [ ]:
# the startstops are nested. we need to unnest this for action stuff only. but use nested for all other
hx_ddf.head(20)

In [ ]:
# the startstops are nested. we need to unnest this for action stuff only.
exploded_df = hx_ddf.explode("startstops")

In [ ]:
# AAAAGH THIS TOOK FOREVER TO FIGURE OUT O_O
exploded_df['action'] = exploded_df['startstops'].apply(lambda x: x['action'], meta = ("action", str))
exploded_df['start'] = exploded_df['startstops'].apply(lambda x: x['start'], meta = ("start", np.float64))
exploded_df['end'] = exploded_df['startstops'].apply(lambda x: x['stop'], meta = ("stop", np.float64))
exploded_df['action_duration'] = exploded_df['end'] - exploded_df['start']

In [ ]:
exploded_df_only_agg_fields = exploded_df[['worker', 'action', 'action_duration']]

In [ ]:
time_per_worker_and_action = exploded_df_only_agg_fields.groupby(['worker','action']).agg("sum")

This is final for time_per_worker_and_action

In [ ]:
time_per_worker_and_action.head(20)

In [ ]:
# now get nbytes per worker

In [ ]:
nbytes_per_worker = hx_ddf[['worker', 'nbytes']].groupby(["worker"]).agg("sum")

In [ ]:
nbytes_per_worker.head(20)